In [ ]:
# Copyright 2023, Acadential, All rights reserved.

# 9-2. Fully Connected Neural Network PyTorch로 구현해보기

## Import Torch library

In [1]:
import torch 
from torch import nn 

## Build a Model

### check device

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


### Define Model Class

Section 2-7에서 살펴보았던 내용인데 다시 Recap해보면 다음과 같습니다.

```nn.Module```을 사용하려면 기본적으로 다음 두 ```Method```을 Override 합니다:
1. ```__init__``` : 신경망 모듈에서 사용되는 모든 모듈을 정의합니다.
2. ```forward``` : 신경망 모듈에서 사용되는 모든 모듈의 연산을 정의합니다.

In [3]:
# 뉴럴네트워크라는 NN.Module 기반의 클래스를 정리하였는데 여기 아래에
# init과 forward 메서드가 각각 정의됩니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # Neural Network을 구성하는 layer들을
        # initialize하는 부분
        pass 

    def forward(self, x):
        # Neural Network의 forward pass을 정의하는 부분
        # 이때 x은 input tensor
        pass 


## Neural Network을 구성할 각 Layer

In [4]:
batch_size = 8

# 디버깅에활용될 샘플데이터 랜덤하게 생성
# 미니배치 by 인풋피처 크기로 행렬을 만들게됨.
# 8 images, 1 channel, 28x28 pixels
sample_data = torch.rand(batch_size, 1, 28, 28)  

In [16]:
# Forward pass
# flatten operation을 통해서 2차원 행렬로 변환해 주었다.
# flatten을 첫번째 dimension부터 적용하라는 의미로 startDim은 1로 지정하였다.

x = torch.flatten(sample_data, start_dim=1)
print("Flattened shape = ", x.shape)

# x shpae은 미니배치 by input feature의 갯수로 28x28이 flatten 되어서
# 28제곱인 784가 된 것을 확인할 수 있습니다.

Flattened shape =  torch.Size([8, 784])


### 참고 사항,
뉴럴 레이어들을 차례대로 쌓게 되는데 출력되는 feature의 크기도 점진적으로 줄여 나가야 함.
점진적으로 출력되는 feature 크기를 줄이는 이유?
1. 맨 마지막에 최종적으로 출력되는 크기가 class의 개수가 되도록 하는 것
2. 출력되는 feature 크기를 줄이는 것은 곧 Layer의 weight 개수를 줄이게 됨 → Model의 Complexity가 너무 커지는 것을 방지할 수 있고 이는 곧 Regularization의 효과
3. Layer을 거듭할수록 풀려는 task와 관련된 추상적인 개념들을 학습하게 됨
   - 예를 들어서 숫자 0은 꺽임이 없다거나 숫자 7은 뾰족하게 튀어나와 있는 특징들과 같은 추상적인 개념들
   - 이러한 추상적인 개념들은 latent feature가 원래의 feature space보다 더 작은 차원의 subspace 상으로 mapping된다고 볼 수 있기 때문에 출력 feature 크기를 줄이는 것 (Manifold 이론)

In [17]:
# First FC Layer(첫번쨰  layer)
fc1 = nn.Linear(784, 784 // 4) #(input,output)
# 이 fc1을 텐서 X에 적용을 하게 되면 출력되는 X의 크기는 8x196이 되는 것을 확인할 수 있다.
# 이 FC layer는 단순히 행렬의 곱이므로 여기는 non-linear가 없다.
x = fc1(x)
print(x.shape)  # torch.Size([8, 196])

torch.Size([8, 196])


In [18]:
# ReLU Layer
# 우리는 nonlinear로 ReLU 레이어를 지정해보겠다.
# 이떄 ReLU는 X에 element wise(원소끼리 계산)되기 때문에 크기 변화없음.
relu = nn.ReLU()
x = relu(x) 
print(x.shape)  # torch.Size([8, 196])

torch.Size([8, 196])


In [19]:
# Second FC + ReLU Layer
# 두번째 레이어는 196 크기의 피처를 또 1/4크기인 49로 축소해주는 FC 레이어가 되겠다.
fc2 = nn.Linear(784 // 4, 784 // 16)
# Forward pass
x = fc2(x)
# apply relu
x = relu(x)
print(x.shape)  # torch.Size([8, 49])

torch.Size([8, 49])


In [20]:
# Third FC + ReLU Layer
fc3 = nn.Linear(784 // 16, 10)
# Forward pass
x = fc3(x)
# apply sigmoid
sigmoid = nn.Sigmoid()
# 시그모이드 함수를 적용함으로써 각 로짓이 0에서 1사이의 값을 가지도록 구성하였다.
x = sigmoid(x)
print(x.shape)  # torch.Size([8, 10])

torch.Size([8, 10])


## Neural Network 정의
앞서서 정의한 각 Layer들로 구성된 Neural Network를 정의합니다.

In [21]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # Neural Network을 구성하는 layer들을
        # initialize하는 부분
        self.fc1 = nn.Linear(784, 784 // 4)
        self.fc2 = nn.Linear(784 // 4, 784 // 16)
        self.fc3 = nn.Linear(784 // 16, 10)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Neural Network의 forward pass을 차례대로 정의하는 부분
        # x은 input tensor
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x


In [22]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (fc1): Linear(in_features=784, out_features=196, bias=True)
  (fc2): Linear(in_features=196, out_features=49, bias=True)
  (fc3): Linear(in_features=49, out_features=10, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [22]:
# 위 방법처럼도 구현할 수 있으나, 더 간단하게 nn.Sequential로 구현할 수 있다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # Neural Network을 구성하는 layer들을
        # initialize하는 부분
        self.fc_layers = nn.Sequential(
            nn.Linear(784, 784 // 4),
            nn.ReLU(),
            nn.Linear(784 // 4, 784 // 16),
            nn.ReLU(),
            nn.Linear(784 // 16, 10),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Neural Network의 forward pass을 정의하는 부분
        # x은 input tensor
        x = torch.flatten(x, start_dim=1)
        x = self.fc_layers(x)
        return x


In [23]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (fc_layers): Sequential(
    (0): Linear(in_features=784, out_features=196, bias=True)
    (1): ReLU()
    (2): Linear(in_features=196, out_features=49, bias=True)
    (3): ReLU()
    (4): Linear(in_features=49, out_features=10, bias=True)
    (5): Sigmoid()
  )
)


## Forward pass

In [31]:
X = torch.rand(8, 1, 28, 28, device=device)
logits = model(X)
print(f"Logits shape: {logits.shape}")
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Logits shape: torch.Size([8, 10])
Predicted class: tensor([2, 2, 2, 2, 2, 2, 2, 2])


In [32]:
logits.shape

torch.Size([8, 10])

In [33]:
# predicted probability 계산을 하려면SOFTMAX함수를 적용할 수 있다.
pred_prob = nn.Softmax(dim=1)(logits)

In [34]:
# argmax는 그 확률 값을 클라스 인덱스 예측 값으로 변환해 주는 것이고
# 여기서 softmax는 logit을 예측 확률 값으로 변환해 주는 연산으로 볼 수 있다.
y_pred = pred_prob.argmax(1)

In [35]:
y_pred

tensor([2, 2, 2, 2, 2, 2, 2, 2])